In [ ]:
%store -r s3_bucket
%store -r capture_upload_path

In [ ]:
results = !aws s3 ls {capture_upload_path} --recursive
processed = []

for result in results:
    partial = result.split()[-1]
    path = f"s3://{s3_bucket}/{partial}"
    processed.append(path)
    
processed

In [ ]:
!mkdir -p captured

In [ ]:
for index, path in enumerate(processed):
    print(index, path)
    !aws s3 cp {path} captured/{index}.jsonl

In [ ]:
import json

def load_json_file(path):
    output = []
    
    with open(path) as f:
        output = [json.loads(line) for line in f]
        
    return output

In [ ]:
all_json = []

for index, _ in enumerate(processed):
    print(f"INDEX: {index}")
    new_records = load_json_file(f"captured/{index}.jsonl")
    all_json = all_json + new_records
    
    
all_json

In [ ]:
!pip3 install flatten-dict

In [ ]:
from flatten_dict import flatten

In [ ]:
first = flatten(all_json[0], reducer='dot')
first

In [ ]:
flattened_json = []

for entry in all_json:
    result = flatten(entry, reducer='dot')
    flattened_json.append(result)
    
flattened_json

In [ ]:
import pandas as pd
df = pd.DataFrame(flattened_json)
df

In [ ]:
df[['x', 'y']] = df['captureData.endpointInput.data'].str.split(',', 1, expand=True)

In [ ]:
df

In [ ]:
df['predicted_label'] = df['captureData.endpointOutput.data'].str.strip()

In [ ]:
df

In [ ]:
clean_df = df[['predicted_label', 'x', 'y']]

In [ ]:
clean_df.head()

In [ ]:
clean_df = clean_df.astype({
    'predicted_label': 'int',
    'x': 'float',
    'y': 'float',
})

In [ ]:
clean_df.head()